# 1. Import the relevant libraries, along with your ords_prods_merge dataframe.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# define the pathname

path = r'/Users/iMac/Library/CloudStorage/OneDrive-Personal/Data Analytics/Data Analytics Immersion/Exercise 4/2024-01_Instacart Basket Analysis'

In [3]:
# import ords_prods_merge DataFrame

df = pd.read_pickle(os.path.join(path, '02 Data', '00 Prepared Data', 'orders_products_checked.pkl'))

# 2. Find the aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe.

In [4]:
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset?

The mean order number for each department in the entire dataframe is slightly different from the mean order number in the subset, which is not surprising, given the limited nature of the subset.

For example, comparing department_id 1: 

    Entire DataFrame: 15.457838
    Subset: 15.577493

Similarly, the mean order numbers for other departments also show some differences between the entire dataframe and the subset, but they are not significantly at variance with one another.

Departments with higher mean values in the subset may not necessarily have the same ranking in the entire dataframe, as the subset might not capture the full diversity of the data.

# 4. Create a loyalty flag for existing customers using the transform() and loc() functions

In [5]:
# create 'max_order' column

df['max_order'] = df.groupby(['user_id'])['order_number'].transform('max')

# create 'loyaly_flag' based on the corresponding 'max_order' value.

df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [6]:
df[['user_id', 'order_number', 'loyalty_flag', 'max_order']].head(10)

,user_id,order_number,loyalty_flag,max_order
0,138,28,Regular customer,32
1,138,30,Regular customer,32
2,709,2,New customer,5
3,764,1,New customer,3
4,764,3,New customer,3
5,777,16,Regular customer,26
6,825,3,New customer,9
7,910,12,Regular customer,12
8,1052,10,Regular customer,20
9,1052,15,Regular customer,20


# 5. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer).

In [7]:
# Basic statistics of product prices for each loyalty category

df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

### It seems that on average, loyal customers tend to purchase products at a slightly lower mean price compared to new and regular customers. However, the high maximum prices suggest the presence of outliers in all loyalty categories.

# 6. Create a spending flag for each user based on the average price across all their orders.

In [8]:
# Create 'spending_flag' column based on the mean spending value

df['spending_freq'] = df.groupby('user_id')['prices'].transform('mean')

# Assign labels based on the spending_flag values

df.loc[df['spending_freq'] < 10, 'spending_flag'] = 'Low spender'
df.loc[df['spending_freq'] >= 10, 'spending_flag'] = 'High spender'

In [9]:
df[['user_id', 'spending_freq', 'spending_flag']].head(10)

,user_id,spending_freq,spending_flag
0,138,6.935811,Low spender
1,138,6.935811,Low spender
2,709,7.930208,Low spender
3,764,4.972414,Low spender
4,764,4.972414,Low spender
5,777,6.935398,Low spender
6,825,5.957576,Low spender
7,910,6.680000,Low spender
8,1052,7.162500,Low spender
9,1052,7.162500,Low spender


# 7. Create an order frequency flag that marks the regularity of a user’s ordering behavior.

In [10]:
# Calculate the median of "days_since_prior_order" for each user

df['order_freq'] = df.groupby('user_id')['days_since_prior_order'].transform('median')

# Create the order frequency flag based on the median

df.loc[df['order_freq'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df.loc[(df['order_freq'] > 10) & (df['order_freq'] <= 20), 'order_frequency_flag'] = 'Regular customer'
df.loc[df['order_freq'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [11]:
df[['user_id', 'order_freq', 'order_frequency_flag']].head(10)

,user_id,order_freq,order_frequency_flag
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
5,777,11.0,Regular customer
6,825,20.0,Regular customer
7,910,6.0,Frequent customer
8,1052,10.0,Frequent customer
9,1052,10.0,Frequent customer


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   orders_day_of_week      int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  spending_freq           float64 
 21  spendi

# 8. Data cleaning form Task 4.9

In [13]:
# determine values above the $100 threshold

df.loc[df['prices'] > 100]

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending_freq,spending_flag,order_freq,order_frequency_flag
13100147,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,912404,17,12,2,14,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
13100148,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,603376,17,22,6,16,...,High-range product,Regularly busy,Regularly busy,Fewest orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
13100149,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,3264360,135,2,2,21,...,High-range product,Regularly busy,Regularly busy,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
13100150,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,892534,135,3,0,8,...,High-range product,Busiest day,Busiest days,Fewest orders,4,New customer,1154.792308,High spender,12.0,Regular customer
13100151,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,229704,342,8,1,19,...,High-range product,Regularly busy,Busiest days,Fewest orders,16,Regular customer,114.426619,High spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21786876,33664,2 % Reduced Fat Milk,84,16,99999.0,2249946,204099,29,0,8,...,High-range product,Busiest day,Busiest days,Fewest orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
21786877,33664,2 % Reduced Fat Milk,84,16,99999.0,2363282,204099,31,0,9,...,High-range product,Busiest day,Busiest days,Fewest orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
21786878,33664,2 % Reduced Fat Milk,84,16,99999.0,3181945,204395,13,3,15,...,High-range product,Regularly busy,Least busy days,Most orders,15,Regular customer,451.153540,High spender,5.0,Frequent customer
21786879,33664,2 % Reduced Fat Milk,84,16,99999.0,2486215,205227,7,3,20,...,High-range product,Regularly busy,Least busy days,Average orders,12,Regular customer,1178.381871,High spender,12.0,Regular customer


In [14]:
# mark outlier values as NaNs

df.loc[df['prices'] >100, 'prices'] = np.nan

In [15]:
# check max price

df['prices'].max()

25.0

# 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [16]:
# Export data to pkl

df.to_pickle(os.path.join(path, '02 Data','00 Prepared Data', 'ords_prods_end.pkl'))